<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports-and-settings" data-toc-modified-id="Imports-and-settings-1">Imports and settings</a></span></li><li><span><a href="#new-features" data-toc-modified-id="new-features-2">new features</a></span><ul class="toc-item"><li><span><a href="#encapsulated-function" data-toc-modified-id="encapsulated-function-2.1">encapsulated function</a></span></li><li><span><a href="#core" data-toc-modified-id="core-2.2">core</a></span></li></ul></li><li><span><a href="#String-execution" data-toc-modified-id="String-execution-3">String execution</a></span></li><li><span><a href="#Standard" data-toc-modified-id="Standard-4">Standard</a></span></li><li><span><a href="#Example" data-toc-modified-id="Example-5">Example</a></span></li></ul></div>

# Imports and settings

In [1]:
from bokeh.io import *
from bokeh.layouts import *
from bokeh.plotting import *
from bokeh.models.renderers import *
from bokeh.palettes import *
from bokeh.models.widgets import *
from bokeh.models import *
from bokeh import events

from NearFieldOptics.Materials import *
from NearFieldOptics import TipModels as T
import copy
import numpy as np
from matplotlib import pyplot as plt

output_notebook()
# output_file("new GUI layout.html")

<material_types.TabulatedMaterialFromFile.__init__>:
	Loading tabulated material data from file "Bi2Se3_epsilon.pickle"...
<material_types.TabulatedMaterialFromFile.__init__>:
	Loading tabulated material data from file "PMMA_epsilon.pickle"...
<material_types.TabulatedMaterialFromFile.__init__>:
	Loading tabulated material data from file "sio2_300nm_extracted_epsilon_cone_A=2a.pickle"...
<material_types.TabulatedMaterialFromFile.__init__>:
	Loading tabulated material data from file "TaS2_eps_230K.csv"...
<material_types.TabulatedMaterialFromFile.__init__>:
	Loading tabulated material data from file "TaS2_eps_30K.csv"...
<material_types.TabulatedMaterialFromFile.__init__>:
	Loading tabulated material data from file "Erik_BSTS_epsilon.pickle"...
<material_types.TabulatedMaterialFromFile.__init__>:
	Loading tabulated material data from file "VO2_Insulating.pickle"...
<material_types.TabulatedMaterialFromFile.__init__>:
	Loading tabulated material data from file "VO2_Metallic.pickle"...


Loading BokehJS ...

# new features

## encapsulated function

In [2]:
def header():
    text = Div(text="<h1><b>The Lightning Rod Model GUI</b></h1>",width=400, height=30)
    title = WidgetBox(text, width=400)

    callback1 = CustomJS(code="alert('Don't drag away the logo!')")
    tap = TapTool(callback=callback1)
    cu = figure(x_range=(0,6), y_range=(-3.5,0.2),width=250,height=60,toolbar_location=None, tools=[tap])
    cu.outline_line_color = "white"
    cu.image_url(url=['Columbia logo.png'], x=0, y=0.2,w=6, h=3.5)
    cu.xgrid.grid_line_color = None
    cu.ygrid.grid_line_color = None
    cu.axis.visible = False

    callback1 = CustomJS(code="alert('Don't drag away the logo!')")
    tap = TapTool(callback=callback1)
    basov = figure(x_range=(0,6), y_range=(-3.5,0.2),width=150,height=60,toolbar_location=None, tools=[tap])
    basov.outline_line_color = "white"
    basov.image_url(url=['basov lab logo.png'], x=0, y=0.2,w=6, h=3.5)
    basov.xgrid.grid_line_color = None
    basov.ygrid.grid_line_color = None
    basov.axis.visible = False
    return [title,cu,basov]

def space():
    empty = Div(text="",width=400, height=1)
    space = WidgetBox(empty, width=400)
    return [empty]

In [3]:
def calculate_and_plot(freqs,sample,quantity,graph):
    if quantity.value=="near-field signal":
        ##no normalization
        calculated=T.LightningRodModel(freqs,rp=sample.reflection_p,a=a,Nqs=Nqs,Nzs=Nzs,\
                                            amplitude=amplitude,normalize_to=None,\
                                            normalize_at=normFrequency)
        global nfs
        nfs=graph.line(freqs,abs(calculated['signal_3']),legend="s3",line_color="#0093dd",line_width=2)
        graph.title.text=exitMaterial.value
        graph.title.align="center"

    elif quantity.value=="far-field reflectivity (p-polarized)":
        ##picked angle 0 as default, generalize
        rp_farfield=sample.reflection_p(freqs,angle=0)
        global fReflecP
        fReflecP=graph.line(freqs,abs(rp_farfield),legend="far field (p) amplitude",line_color="#0093dd",line_width=2)
        graph.title.text=exitMaterial.value
        graph.title.align="center"

    elif quantity.value=="far-field reflectivity (s-polarized)":
        ##picked angle 0 as default, generalize
        rs_farfield=sample.reflection_s(freqs,angle=0)
        global fReflecS
        fReflecS=graph.line(freqs,abs(rs_farfield),legend="far field (s) amplitude",line_color="#0093dd",line_width=2)
        graph.title.text=exitMaterial.value
        graph.title.align="center"

    #need to add far-field transmission

    elif quantity.value=="permittivity":
        eps=sample.epsilon(freqs)
        global rl
        rl=graph.line(freqs,eps.real,legend="real",line_color="#0093dd",line_width=2)
        global img
        img=graph.line(freqs,eps.imag,legend="imaginary",line_color="orange",line_width=2)
        graph.title.text=exitMaterial.value
        graph.title.align="center"

    elif quantity.value=="near-field reflectivity":
        rp_nearfield=sample.reflection_p(freqs,angle=None,q=1/a)
        global nReflec
        nReflec=graph.line(freqs,abs(rp_nearfield),legend="near field amplitude",line_color="orange",line_width=2)
        graph.title.text=exitMaterial.value
        graph.title.align="center"

    else:
        #plot a thick red cross on panel
        x1 = np.linspace(0, 10, 100)
        y1 = x1
        y2 = 10-x1
        global x
        x=graph.line(x1, y1, legend="ERROR", line_color="red", line_width=6)
        global xx
        xx=graph.line(x1, y2, legend="ERROR", line_color="red", line_width=6)
        graph.title.text=exitMaterial.value
        graph.title.align="center"

## core

In [4]:
def modify_doc(doc):
    
    #graphs
    s1 = figure(x_axis_label='frequency',plot_width=500, plot_height=300)
    s1.xgrid.grid_line_color = None
    s1.ygrid.grid_line_color = None

    s2 = figure(x_axis_label='frequency',plot_width=500, plot_height=300)
    s2.xgrid.grid_line_color = None
    s2.ygrid.grid_line_color = None

    s3 = figure(x_axis_label='frequency',plot_width=500, plot_height=300)
    s3.xgrid.grid_line_color = None
    s3.ygrid.grid_line_color = None

    s4 = figure(x_axis_label='frequency',plot_width=500, plot_height=300)
    s4.xgrid.grid_line_color = None
    s4.ygrid.grid_line_color = None
    
    #widgets
    entranceMaterial = Select(title="Entrance material:", value="Air", options=[
        "Air", "SiO2_300nm", "Si", "BN_Caldwell", "BN_GPR", ""])
    exitMaterial = Select(title="Exit material:", value="Si_80K", options=["Air","Si_80K", "Si_150K", "Si_300K", "SiO2"])
    
    thickness = TextInput(title="Thickness (nm): ",value="0")
    add = Button(label="Add a layer on top", button_type="primary")  
    
    size = TextInput(title="Tip apex radius size (nm): ",value="10")
    oscAmplitude = TextInput(title="Tip oscillation amplitude (nm): ",value="80")

    freqNum = TextInput(title="# of frequency points: ",value="256")
    numQ = TextInput(title="# of q points: ",value="244")
    numZ = TextInput(title="# of z points: ",value="40")

    normFreq = TextInput(title="Normalization frequency (cm^-1): ",value="0")
    normMat = Select(title="Normalization material:", value="Au", options=["Au","Si_80K", "Si_150K", "Si_300K", "SiO2"])

    quantity = Select(title="Quantity to calculate:", value="permittivity", options=[ 
                    "far-field reflectivity (p-polarized)", "far-field reflectivity (s-polarized)","far-field transmission",
                    "near-field reflectivity","near-field signal","permittivity"])
    
    plot_panel = RadioButtonGroup(
        labels=["Plot in 1st panel", "Plot in 2nd panel", "Plot in 3rd panel","Plot in 4th panel"], active=0)
    
    run = Button(label="Run!", button_type="success")
    clear = Button(label="Clear graph",button_type="warning")
    
    #interact
    def update_parameters():
        
        sample=TabulatedMaterialFromFile(exitMaterial.value+'.csv')
        thick=float(thickness.value)*1e-9
        a=float(size.value)*1e-9
        amplitude=float(oscAmplitude.value)*1e-9
        freqNumber=int(freqNum.value)
        Nqs=int(numQ.value)
        Nzs=int(numZ.value)
        normFrequency=float(normFreq.value)
        normMaterial=normMat.value

        f0=sample._eps_data.axes[0]
        freqs=np.linspace(f0.min(),f0.max(),freqNumber)
        
        if plot_panel.value=="Plot in 1st panel":
            graph=s1
        elif plot_panel.value=="Plot in 2nd panel":
            graph=s2
        elif plot_panel.value=="Plot in 3rd panel":
            graph=s3
        elif plot_panel.value=="Plot in 4th panel":
            plot_graph=s4
        else:
            graph=s1
        
#         calculate_and_plot(freqs,sample,quantity,graph)
        if quantity.value=="near-field signal":
            ##no normalization
            calculated=T.LightningRodModel(freqs,rp=sample.reflection_p,a=a,Nqs=Nqs,Nzs=Nzs,\
                                                amplitude=amplitude,normalize_to=None,\
                                                normalize_at=normFrequency)
            global nfs
            nfs=graph.line(freqs,abs(calculated['signal_3']),legend="s3",line_color="#0093dd",line_width=2)
            graph.title.text=exitMaterial.value
            graph.title.align="center"

        elif quantity.value=="far-field reflectivity (p-polarized)":
            ##picked angle 0 as default, generalize
            rp_farfield=sample.reflection_p(freqs,angle=0)
            global fReflecP
            fReflecP=graph.line(freqs,abs(rp_farfield),legend="far field (p) amplitude",line_color="#0093dd",line_width=2)
            graph.title.text=exitMaterial.value
            graph.title.align="center"

        elif quantity.value=="far-field reflectivity (s-polarized)":
            ##picked angle 0 as default, generalize
            rs_farfield=sample.reflection_s(freqs,angle=0)
            global fReflecS
            fReflecS=graph.line(freqs,abs(rs_farfield),legend="far field (s) amplitude",line_color="#0093dd",line_width=2)
            graph.title.text=exitMaterial.value
            graph.title.align="center"

        #need to add far-field transmission

        elif quantity.value=="permittivity":
            eps=sample.epsilon(freqs)
            global rl
            rl=graph.line(freqs,eps.real,legend="real",line_color="#0093dd",line_width=2)
            global img
            img=graph.line(freqs,eps.imag,legend="imaginary",line_color="orange",line_width=2)
            graph.title.text=exitMaterial.value
            graph.title.align="center"

        elif quantity.value=="near-field reflectivity":
            rp_nearfield=sample.reflection_p(freqs,angle=None,q=1/a)
            global nReflec
            nReflec=graph.line(freqs,abs(rp_nearfield),legend="near field amplitude",line_color="orange",line_width=2)
            graph.title.text=exitMaterial.value
            graph.title.align="center"

        else:
            #plot a thick red cross on panel
            x1 = np.linspace(0, 10, 100)
            y1 = x1
            y2 = 10-x1
            global x
            x=graph.line(x1, y1, legend="ERROR", line_color="red", line_width=6)
            global xx
            xx=graph.line(x1, y2, legend="ERROR", line_color="red", line_width=6)
            graph.title.text=exitMaterial.value
            graph.title.align="center"

    def clear_graph():
        if plot_panel.value=="Plot in 1st panel":
            graph=s1
        elif plot_panel.value=="Plot in 2nd panel":
            graph=s2
        elif plot_panel.value=="Plot in 3rd panel":
            graph=s3
        elif plot_panel.value=="Plot in 4th panel":
            graph=s4
        else:
            graph=s1
        
        if quantity.value=="near-field signal":
            graph.renderers.remove(nfs)
            graph.title.remove()
            graph.legend.remove()
        elif quantity.value=="far-field reflectivity (p-polarized)":
            graph.renderers.remove(fReflecP)
        elif quantity.value=="far-field reflectivity (s-polarized)":
            graph.renderers.remove(fReflecS)
        elif quantity.value=="permittivity":
            graph.renderers.remove(rl)
            graph.renderers.remove(img)
        elif quantity.value=="near-field reflectivity":
            graph.renderers.remove(nReflec)
        else:
            graph.renderers.remove(x)
            graph.renderers.remove(xx)

    run.on_click(update_parameters)
    clear.on_click(clear_graph)
    
    #layout
    wb1 = WidgetBox(entranceMaterial,exitMaterial,thickness,add,width=200)
    wb2 = [WidgetBox(size,oscAmplitude,width=250),
           WidgetBox(freqNum,numQ,numZ,width=250),
           WidgetBox(normFreq,normMat,width=250),
           WidgetBox(plot_panel,width=300),
           WidgetBox(quantity,run, clear,width=250)]
    
    l = layout(header(),
               [space()],
               [wb1,[[s1,s3],[s2,s4]]],
               wb2)
    doc.add_root(l)
    
show(modify_doc)

# String execution

In [5]:
exec('SingleLayerGraphene()')

# Standard

In [6]:
def modify_doc(doc):
    
    #FIRST LAYER        
    def header():
        text = Div(text="<h1><b>The Lightning Rod Model GUI</b></h1>",width=400, height=30)
        title = WidgetBox(text, width=400)
        
        callback1 = CustomJS(code="alert('Don't drag away the logo!')")
        tap = TapTool(callback=callback1)
        cu = figure(x_range=(0,6), y_range=(-3.5,0.2),width=250,height=60,toolbar_location=None, tools=[tap])
        cu.image_url(url=['Columbia logo.png'], x=0, y=0.2,w=6, h=3.5)
        cu.xgrid.grid_line_color = None
        cu.ygrid.grid_line_color = None
        cu.axis.visible = False
        
        callback1 = CustomJS(code="alert('Don't drag away the logo!')")
        tap = TapTool(callback=callback1)
        basov = figure(x_range=(0,6), y_range=(-3.5,0.2),width=200,height=60,toolbar_location=None, tools=[tap])
        basov.image_url(url=['basov lab logo.png'], x=0, y=0.2,w=6, h=3.5)
        basov.xgrid.grid_line_color = None
        basov.ygrid.grid_line_color = None
        basov.axis.visible = False
        return [title,cu,basov]
    
    def space():
        empty = Div(text="",width=400, height=1)
        space = WidgetBox(empty, width=400)
        return [empty]
    
    #graphs
    s1 = figure(x_axis_label='frequency',plot_width=500, plot_height=300)
    s1.xgrid.grid_line_color = None
    s1.ygrid.grid_line_color = None

    s2 = figure(x_axis_label='frequency',plot_width=500, plot_height=300)
    s2.xgrid.grid_line_color = None
    s2.ygrid.grid_line_color = None

    s3 = figure(x_axis_label='frequency',plot_width=500, plot_height=300)
    s3.xgrid.grid_line_color = None
    s3.ygrid.grid_line_color = None

    s4 = figure(x_axis_label='frequency',plot_width=500, plot_height=300)
    s4.xgrid.grid_line_color = None
    s4.ygrid.grid_line_color = None
    
    #widgets
    matDim = RadioButtonGroup(labels=["2D", "3D"], active=0)
    subMat = Select(title="Substrate material:", value="Si_80K", options=["Si_80K", "Si_150K", "Si_300K", "SiO2"])
    thickness = TextInput(title="Thickness (nm): ",value="0")
    add = Button(label="Add a layer on top", button_type="primary")  
    
    size = TextInput(title="Tip apex radius size (nm): ",value="10")
    oscAmplitude = TextInput(title="Tip oscillation amplitude (nm): ",value="80")

    freqNum = TextInput(title="# of frequency points: ",value="256")
    numQ = TextInput(title="# of q points: ",value="244")
    numZ = TextInput(title="# of z points: ",value="40")

    normFreq = TextInput(title="Normalization frequency (cm^-1): ",value="0")
    normMat = Select(title="Normalization material:", value="Au", options=["Au","Si_80K", "Si_150K", "Si_300K", "SiO2"])

    quantity = Select(title="Quantity to calculate:", value="permittivity", options=[ 
                    "far-field reflectivity (p-polarized)", "far-field reflectivity (s-polarized)","far-field transmission",
                    "near-field reflectivity","near-field signal","permittivity"])
        
    plot_panel = Select(title="Plot panel",value="Plot in 1st panel", options=["Plot in 1st panel", 
            "Plot in 2nd panel", "Plot in 3rd panel", "Plot in 4th panel"])
    
    run = Button(label="Run!", button_type="success")
    clear = Button(label="Clear graph",button_type="warning")
    
    #interact
    def update_parameters():
        
        sample=TabulatedMaterialFromFile(subMat.value+'.csv')
        thick=float(thickness.value)*1e-9
        a=float(size.value)*1e-9
        amplitude=float(oscAmplitude.value)*1e-9
        freqNumber=int(freqNum.value)
        Nqs=int(numQ.value)
        Nzs=int(numZ.value)
        normFrequency=float(normFreq.value)
        normMaterial=normMat.value

        f0=sample._eps_data.axes[0]
        freqs=np.linspace(f0.min(),f0.max(),freqNumber)
        
        if plot_panel.value=="Plot in 1st panel":
            graph=s1
        elif plot_panel.value=="Plot in 2nd panel":
            graph=s2
        elif plot_panel.value=="Plot in 3rd panel":
            graph=s3
        elif plot_panel.value=="Plot in 4th panel":
            plot_graph=s4
        else:
            graph=s1
        
        if quantity.value=="near-field signal":
            ##no normalization
            calculated=T.LightningRodModel(freqs,rp=sample.reflection_p,a=a,Nqs=Nqs,Nzs=Nzs,\
                                                amplitude=amplitude,normalize_to=None,\
                                                normalize_at=normFrequency)
            global nfs
            nfs=graph.line(freqs,abs(calculated['signal_3']),legend="s3",line_color="#0093dd",line_width=2)
            graph.title.text=subMat.value
            graph.title.align="center"

        elif quantity.value=="far-field reflectivity (p-polarized)":
            ##picked angle 0 as default, generalize
            rp_farfield=sample.reflection_p(freqs,angle=0)
            global fReflecP
            fReflecP=graph.line(freqs,abs(rp_farfield),legend="far field (p) amplitude",line_color="#0093dd",line_width=2)
            graph.title.text=subMat.value
            graph.title.align="center"

        elif quantity.value=="far-field reflectivity (s-polarized)":
            ##picked angle 0 as default, generalize
            rs_farfield=sample.reflection_s(freqs,angle=0)
            global fReflecS
            fReflecS=graph.line(freqs,abs(rs_farfield),legend="far field (s) amplitude",line_color="#0093dd",line_width=2)
            graph.title.text=subMat.value
            graph.title.align="center"

        #need to add far-field transmission

        elif quantity.value=="permittivity":
            eps=sample.epsilon(freqs)
            global rl
            rl=graph.line(freqs,eps.real,legend="real",line_color="#0093dd",line_width=2)
            global img
            img=graph.line(freqs,eps.imag,legend="imaginary",line_color="orange",line_width=2)
            graph.title.text=subMat.value
            graph.title.align="center"

        elif quantity.value=="near-field reflectivity":
            rp_nearfield=sample.reflection_p(freqs,angle=None,q=1/a)
            global nReflec
            nReflec=graph.line(freqs,abs(rp_nearfield),legend="near field amplitude",line_color="orange",line_width=2)
            graph.title.text=subMat.value
            graph.title.align="center"

        else:
            #plot a thick red cross on panel
            x1 = np.linspace(0, 10, 100)
            y1 = x1
            y2 = 10-x1
            global x
            x=graph.line(x1, y1, legend="ERROR", line_color="red", line_width=6)
            global xx
            xx=graph.line(x1, y2, legend="ERROR", line_color="red", line_width=6)
            graph.title.text=subMat.value
            graph.title.align="center"

    def clear_graph():
        if plot_panel.value=="Plot in 1st panel":
            graph=s1
        elif plot_panel.value=="Plot in 2nd panel":
            graph=s2
        elif plot_panel.value=="Plot in 3rd panel":
            graph=s3
        elif plot_panel.value=="Plot in 4th panel":
            graph=s4
        else:
            graph=s1

        if quantity.value=="near-field signal":
            graph.renderers.remove(nfs)
        elif quantity.value=="far-field reflectivity (p-polarized)":
            graph.renderers.remove(fReflecP)
        elif quantity.value=="far-field reflectivity (s-polarized)":
            graph.renderers.remove(fReflecS)
        elif quantity.value=="permittivity":
            graph.renderers.remove(rl)
            graph.renderers.remove(img)
        elif quantity.value=="near-field reflectivity":
            graph.renderers.remove(nReflec)
        else:
            graph.renderers.remove(x)
            graph.renderers.remove(xx)

    run.on_click(update_parameters)
    clear.on_click(clear_graph)
    
    #layout
    wb1 = WidgetBox(matDim,subMat,thickness,add,width=200)
    wb2 = [WidgetBox(size,oscAmplitude,width=250),WidgetBox(freqNum,numQ,numZ,width=250),WidgetBox(normFreq,normMat,width=250),WidgetBox(plot_panel,width=250),WidgetBox(quantity,run, clear,width=250)]
    
    l = layout(header(),
               [space()],
               [wb1,[[s1,s3],[s2,s4]]],
               wb2)
    doc.add_root(l)
    
show(modify_doc) 

# Example

In [7]:
import numpy as np

from bokeh.io import curdoc
from bokeh.layouts import row, column
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import Slider, TextInput
from bokeh.plotting import figure

def modify_doc(doc):
    # Set up data
    N = 200
    x = np.linspace(0, 4*np.pi, N)
    y = np.sin(x)
    source = ColumnDataSource(data=dict(x=x, y=y))


    # Set up plot
    plot = figure(plot_height=400, plot_width=400, title="my sine wave",
                  tools="crosshair,pan,reset,save,wheel_zoom",
                  x_range=[0, 4*np.pi], y_range=[-2.5, 2.5])

    plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)


    # Set up widgets
    text = TextInput(title="title", value='my sine wave')
    offset = Slider(title="offset", value=0.0, start=-5.0, end=5.0, step=0.1)
    amplitude = Slider(title="amplitude", value=1.0, start=-5.0, end=5.0, step=0.1)
    phase = Slider(title="phase", value=0.0, start=0.0, end=2*np.pi)
    freq = Slider(title="frequency", value=1.0, start=0.1, end=5.1, step=0.1)


    # Set up callbacks
    def update_title(attrname, old, new):
        plot.title.text = text.value

    text.on_change('value', update_title)

    def update_data(attrname, old, new):

        # Get the current slider values
        a = amplitude.value
        b = offset.value
        w = phase.value
        k = freq.value

        # Generate the new curve
        x = np.linspace(0, 4*np.pi, N)
        y = a*np.sin(k*x + w) + b

        source.data = dict(x=x, y=y)

    for w in [offset, amplitude, phase, freq]:
        w.on_change('value', update_data)


    # Set up layouts and add to document
    inputs = column(text, offset, amplitude, phase, freq)

    doc.add_root(row(inputs, plot, width=800))
show(modify_doc) 


/Users/leo/anaconda3/lib/python3.6/site-packages/bokeh/util/serialization.py:487: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  array = np.fromstring(b64, dtype=data['dtype'])
